<a href="https://colab.research.google.com/github/brancatellimat/speech-emotion-recognition/blob/main/DataPreparation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Preparation
This script use opendatasets library to download RAVDESS and CREMA-D datasets.
The output is a csv file with two columns:

1.   Emotion (label)
2.   File path



### Import libraries

In [ ]:
!pip install opendatasets
import opendatasets

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf


import os
import sys

# librosa is a Python library for analyzing audio and music. It can be used to extract the data from the audio files we will see it later.
import librosa
import librosa.display
import librosa.feature as libf
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, OneHotEncoder, minmax_scale, scale
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split

# to play the audio files
from IPython.display import Audio

import keras
from keras.callbacks import ReduceLROnPlateau
from keras.models import Sequential
from keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, BatchNormalization
from keras.callbacks import ModelCheckpoint

import warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")
warnings.filterwarnings("ignore", category=DeprecationWarning)

### Prepare RAVDESS dataset

In [ ]:
ravdess = '/content/drive/MyDrive/Colab Notebooks/Audio Pattern Recognition/Project_Brancatelli/RAVDESS'

ravdess_emotion_dictionary = {
    1: 'neutral',
    2: 'calm',
    3: 'happy',
    4: 'sad',
    5: 'angry',
    6: 'fear',
    7: 'disgust',
    8: 'surprise'
}

ravdess_directory_list = os.listdir(ravdess)

file_emotion = []
file_path = []

for dir in ravdess_directory_list:
  # There are 20 actors -> we need to extract files for each of them
  actor_directory = os.listdir(ravdess + '/' + dir)
  for file in actor_directory:
    # RAVDESS File name structure: [Modality]_[VocalChannel]_[Emotion]_[Intensity]_[Statement]_[Repetition]_[Actor].wav
    part = file.split('.')[0]
    part = part.split('-')[2]
    file_emotion.append(ravdess_emotion_dictionary.get(int(part)))
    file_path.append(ravdess + '/' + dir + '/' + file)

# Dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])
# Dataframe for path of files
path_df = pd.DataFrame(file_path, columns=['Path'])
Ravdess_df = pd.concat([emotion_df, path_df], axis=1)
Ravdess_df.head()

,Emotions,Path
0,angry,/content/drive/MyDrive/Colab Notebooks/Audio P...
1,surprise,/content/drive/MyDrive/Colab Notebooks/Audio P...
2,disgust,/content/drive/MyDrive/Colab Notebooks/Audio P...
3,surprise,/content/drive/MyDrive/Colab Notebooks/Audio P...
4,disgust,/content/drive/MyDrive/Colab Notebooks/Audio P...


In [ ]:
Ravdess_df = Ravdess_df.drop(Ravdess_df[Ravdess_df['Emotions'] == 'calm'].index)

In [ ]:
Ravdess_df = Ravdess_df.drop(Ravdess_df[Ravdess_df['Emotions'] == 'surprise'].index)

In [ ]:
print(Ravdess_df.Emotions.value_counts())

angry      192
disgust    192
sad        192
happy      192
fear       192
neutral     96
Name: Emotions, dtype: int64


### Prepare CREMA-D dataset

In [ ]:
cremad = '/content/drive/MyDrive/Colab Notebooks/Audio Pattern Recognition/Project_Brancatelli/CREMAD'

crema_emotion_dictionary = {
    'SAD': 'sad',
    'ANG': 'angry',
    'DIS': 'disgust',
    'FEA': 'fear',
    'HAP': 'happy',
    'NEU': 'neutral'
}

crema_directory_list = os.listdir(cremad)

file_emotion = []
file_path = []

for file in crema_directory_list:
  file_path.append(cremad + '/' + file)
  # CREMA file name structure: [ActorID]_[Sentence]_[Emotion]_[Level].wav
  part = file.split('.')[0]
  part = part.split('_')[2]
  file_emotion.append(crema_emotion_dictionary.get(part))

# Dataframe for emotion of files
emotion_df = pd.DataFrame(file_emotion, columns=['Emotions'])
# Dataframe for path of files
path_df = pd.DataFrame(file_path, columns=['Path'])
Crema_df = pd.concat([emotion_df, path_df], axis=1)
Crema_df.head()

,Emotions,Path
0,happy,/content/drive/MyDrive/Colab Notebooks/Audio P...
1,happy,/content/drive/MyDrive/Colab Notebooks/Audio P...
2,disgust,/content/drive/MyDrive/Colab Notebooks/Audio P...
3,happy,/content/drive/MyDrive/Colab Notebooks/Audio P...
4,neutral,/content/drive/MyDrive/Colab Notebooks/Audio P...


### Combine them to get a single dataset

In [ ]:
# Concat all dataframes created so far
data_path = pd.concat([Ravdess_df, Crema_df], axis=0)
data_path.to_csv("apr_ser_data.csv", index=False)
data_path.head()

,Emotions,Path
0,angry,/content/drive/MyDrive/Colab Notebooks/Audio P...
2,disgust,/content/drive/MyDrive/Colab Notebooks/Audio P...
4,disgust,/content/drive/MyDrive/Colab Notebooks/Audio P...
5,angry,/content/drive/MyDrive/Colab Notebooks/Audio P...
7,sad,/content/drive/MyDrive/Colab Notebooks/Audio P...


In [ ]:
print(data_path.Emotions.value_counts())

angry      1463
disgust    1463
sad        1463
happy      1463
fear       1463
neutral    1183
Name: Emotions, dtype: int64
